In [1]:
!pip install streamlit

     |████████████████████████████████| 7.2MB 6.8MB/s 
     |████████████████████████████████| 112kB 54.5MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 4.4MB 48.7MB/s 
     |████████████████████████████████| 122kB 51.3MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=4f344fc08b48c638cfa1db040637af803c597921e61b7983bf380aadd39cf99f
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=599932bd60d4bf329bb97beeae127a445adf610d2820df8c459ab385bf8ec1af
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=5c37dbc0f3173a18313bc441cc707c3476f1c41283980e39b2f0e7c0feafc279
  Stored in directory: /root/.cache/pip/wheel

In [2]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.12-cp36-none-any.whl size=16810 sha256=533e2f9ee8074998442f2263c5d69976f3b3fe81a0f2864581b1305b8e585f81
  Stored in directory: /root/.cache/pip/wheels/69/c3/d6/6968dd4d831794d41c311be1d7af6f4ac151c5d3bd0e6efab8
Successfully built pyngrok


In [48]:
!wget https://cdn.nohat.cc/thumb/f/720/4880279604822016.jpg -o "bg.jpg"

In [3]:
!wget https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt

--2020-09-12 16:11:25--  https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  21.17K  --.-KB/s    in 0.003s  

2020-09-12 16:11:25 (6.12 MB/s) - ‘imagenet_classes.txt’ saved [21675/21675]



In [49]:
%%writefile main.py
import streamlit as st
PAGE_CONFIG = {"page_title":"Image Classifier.io", "page_icon":":smiley:", "layout":"centered"}
st.beta_set_page_config(**PAGE_CONFIG)
import time
import torch
import torchvision.models as models
from PIL import Image
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]
st.set_option('deprecation.showfileUploaderEncoding', False)
html_temp = '''
    <div>
    <center><h1>Image Classifier</h1></center>
    <h1></h1>
    </div>
    '''
st.markdown(html_temp, unsafe_allow_html=True)
import base64
 
def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()
 
def set_png_as_page_bg(jpg_file):
    bin_str = get_base64_of_bin_file(jpg_file)
    page_bg_img = '''
    <style>
    body {
    background-image: url("data:image/jpg;base64,%s");
    background-size: cover;
    }
    </style>
    ''' % bin_str
    st.markdown(page_bg_img, unsafe_allow_html=True)
    return
 
set_png_as_page_bg('bg.jpg')
 
def classify(model, image, classes, model_select):
  model.eval()
  from torchvision import transforms
  
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(image)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
 
  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
  with torch.no_grad():
    output = model(input_batch)
  _, index = torch.max(output, 1)
  percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
  
  with st.spinner('Classifying... Please Wait!'):
    time.sleep(3)
  
  st.write("Hey!", model_select,"has classified your image as a",classes[index[0]],"with confidence",percentage[index[0]].item())


select =  st.sidebar.selectbox('',('Home','Image Classifier'))
progress_bar = st.sidebar.progress(0)
status_text = st.sidebar.empty()
for i in range(1,101):
  status_text.text("%i%% Complete" % i)
  progress_bar.progress(i)
  time.sleep(0.04)
progress_bar.empty()
time.sleep(1.5)
status_text.empty()

if select == 'Image Classifier':
    model_select = st.sidebar.selectbox('Please select a Model from the list',('','AlexNet', 'DenseNet', 'ResNet','VGG'))
    html_temp = '''
    <div>
    <center><h3>Please enter Image for Classification</h3></center>
    </div>
    '''
    st.markdown(html_temp, unsafe_allow_html=True)
    opt = st.selectbox("How do you want to upload the image for classification?\n", ('Please Select', 'Upload image via link', 'Upload image from device'))
 
    if opt == 'Upload image from device':
    
        file = st.file_uploader('Select', type = ['jpg', 'png'])
        
        st.set_option('deprecation.showfileUploaderEncoding', False)
        if file is not None:
            image = Image.open(file)
            
            if model_select == 'VGG':
              model = models.vgg16(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
            elif model_select == 'AlexNet':
              model = models.alexnet(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
            elif model_select == 'DenseNet':
              model = models.densenet121(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
            elif model_select == 'ResNet':
              model = models.resnet101(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
    if opt == 'Upload image via link':
        img = st.text_input('Enter the Image Address')
        if st.button('Submit'):
            import urllib.request
            image = Image.open(urllib.request.urlopen(img))
            if model_select == 'VGG':
              model = models.vgg16(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
            elif model_select == 'AlexNet':
              model = models.alexnet(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
            elif model_select == 'DenseNet':
              model = models.densenet121(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
            elif model_select == 'ResNet':
              model = models.resnet101(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes, model_select)
            
 
    if opt == "Please Select":
        pass

if select == 'Home':
    
    html_temp = '''
    <div style = "background-color: rgba(25,25,112,0.20); padding: 20px; padding-bottom: 30px; padding-left: 20px; padding-right: 20px">
    <center><h2>Hey there!!! Welcome! &#x1F609;</h2></center>
    <h1></h1>
    <center><h3>Enter any image of your choice and our models will classify it for you.</h3></center>
    <h3></h3>
    <center><h4>To know more about Image Classifier with PyTorch click <a href = "https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5"  target="_blank">here</a>.</h4></center> 
    </div>
    '''
    st.markdown(html_temp, unsafe_allow_html=True)
 

Overwriting main.py


In [57]:
!ls

4880279604822016.jpg	bg.jpg		      main.py
4880279604822016.jpg.1	imagenet_classes.txt  sample_data


In [58]:
!ngrok authtoken 1hPyhE9nRAYmmSQvBywqPgm8G1E_3EDf1dzjuxgU4ht7RpD4H

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [59]:
from pyngrok import ngrok

In [66]:
!streamlit run main.py&>/dev/null&

In [67]:
!pgrep streamlit

204
249
300
334
384
397
468
502
518


In [68]:
publ_url = ngrok.connect(port='8501')

In [69]:
publ_url

'http://b4c7b1f769a8.ngrok.io'

In [64]:
#if no of tunnels exceeds 4
ngrok.kill()